# CD1 P1 FT2

### Import Packages

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

### Load Dataset

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
# Define paths to training and validation datasets
data_dir = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\DatasetConv'

# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Aris_Work\AppData\Local\Temp\ipykernel_29528\2621384392.py:2: SyntaxWarning: invalid escape sequence '\P'
  data_dir = 'C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\DatasetConv'


Found 24997 files belonging to 2 classes.
Using 19998 files for training.
Found 24997 files belonging to 2 classes.
Using 4999 files for validation.


### Preprocessing & Augmentation

In [4]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [6]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load the Model with the Trained Model with the Trained Head

In [ ]:
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\StarredModels\\CD_1P1_2epochs_head_epoch002_val0.0556.keras'
model = tf.keras.models.load_model(filepath)

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Aris_Work\AppData\Local\Temp\ipykernel_29528\1346541303.py:1: SyntaxWarning: invalid escape sequence '\P'
  filepath = 'C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\StarredModels\CD_1P1_2epochs_head_epoch002_val0.0556.keras'


### Unfreeze Last VGG Block & Verify

In [8]:
# Unfreeze the last few layers (e.g., last 4 layers)
for layer in model.layers[-4:]:
    layer.trainable = True

# Optionally, print trainable status
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

Layer 0: vgg16, Trainable: True
Layer 1: flatten, Trainable: True
Layer 2: dense, Trainable: True
Layer 3: dropout, Trainable: True
Layer 4: dense_1, Trainable: True


In [9]:
for layer in model.layers[0].layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
def print_model_layers(model, indent=0):
    for layer in model.layers:
        print(" " * indent + f"- {layer.name} ({layer.__class__.__name__}), Trainable: {layer.trainable}")
        # If this layer has sublayers (like Functional or Sequential models)
        if hasattr(layer, 'layers'):
            print_model_layers(layer, indent + 2)

print_model_layers(model)

print(model.optimizer.get_config())

- vgg16 (Functional), Trainable: True
  - input_layer_1 (InputLayer), Trainable: False
  - block1_conv1 (Conv2D), Trainable: False
  - block1_conv2 (Conv2D), Trainable: False
  - block1_pool (MaxPooling2D), Trainable: False
  - block2_conv1 (Conv2D), Trainable: False
  - block2_conv2 (Conv2D), Trainable: False
  - block2_pool (MaxPooling2D), Trainable: False
  - block3_conv1 (Conv2D), Trainable: False
  - block3_conv2 (Conv2D), Trainable: False
  - block3_conv3 (Conv2D), Trainable: False
  - block3_pool (MaxPooling2D), Trainable: False
  - block4_conv1 (Conv2D), Trainable: False
  - block4_conv2 (Conv2D), Trainable: False
  - block4_conv3 (Conv2D), Trainable: False
  - block4_pool (MaxPooling2D), Trainable: False
  - block5_conv1 (Conv2D), Trainable: True
  - block5_conv2 (Conv2D), Trainable: True
  - block5_conv3 (Conv2D), Trainable: True
  - block5_pool (MaxPooling2D), Trainable: False
- flatten (Flatten), Trainable: True
- dense (Dense), Trainable: True
- dropout (Dropout), Trainabl

### Compile

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=3*1e-6)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

### Train & Save Results

In [ ]:
# Define the checkpoint path
model_name = 'CD1 P1_FT2'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\CD1'
checkpoint_path = f"{path}/{model_name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

# Reduce Learning Rate on Plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',       # Watch validation loss
    factor=0.1,               # Reduce LR by 10x
    patience=2,               # Wait for 3 epochs with no improvement
    verbose=1,                # Print when LR is reduced
    min_lr=1e-6               # Don't go below this LR
)

# Stop training early if no improvement
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,                 # stop if no improvement for 5 epochs
    restore_best_weights=True,  # load back best weights automatically
    verbose=1
)

In [13]:
history = model.fit(
    train_dataset,
    epochs=7,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9688 - loss: 0.0844
Epoch 1: saving model to SavedModels/CD_1P1_best_ft2_epoch001_val0.0562.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3430s 5s/step - accuracy: 0.9688 - loss: 0.0844 - val_accuracy: 0.9868 - val_loss: 0.0562
Epoch 2/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9768 - loss: 0.0659
Epoch 2: saving model to SavedModels/CD_1P1_best_ft2_epoch002_val0.0598.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3408s 5s/step - accuracy: 0.9768 - loss: 0.0659 - val_accuracy: 0.9850 - val_loss: 0.0598
Epoch 3/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9784 - loss: 0.0581
Epoch 3: saving model to SavedModels/CD_1P1_best_ft2_epoch003_val0.0648.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3415s 5s/step - accuracy: 0.9784 - loss: 0.0581 - val_accuracy: 0.9860 - val_loss: 0.0648
Epoch 4/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9799 - loss: 0.0538
Epoch 4: saving model to SavedModels/CD_1P1_best_ft2_epoch004_val0.0588.k

In [ ]:
import json

model_name = 'CD1_P1_FT2'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData'
filepath = f"{path}\\{model_name}.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)

### Load Last Model

In [ ]:
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\CD1\\CD1_P1_FT2_007_val0.0577.keras'
model = tf.keras.models.load_model(filepath)

### Compile Again with lower learning rate

In [ ]:
# Define the checkpoint path
model_name = 'CD1 P1_FT2_continue'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\CD1'
checkpoint_path = f"{path}/{model_name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

In [15]:
from tensorflow.keras.optimizers import SGD
optimizer = SGD(
    learning_rate=1e-6,  # 🔥 Very low LR for polish phase (safe & smooth)
    momentum=0.9,        # ✅ Momentum helps smooth updates (standard is 0.9)
    nesterov=True        # 🔥 Nesterov often gives slightly better convergence
)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history2 = model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9856 - loss: 0.0371
Epoch 1: saving model to SavedModels/CD_1P1_best_ft2_epoch001_val0.0589.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3954s 6s/step - accuracy: 0.9856 - loss: 0.0371 - val_accuracy: 0.9876 - val_loss: 0.0589
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9845 - loss: 0.0428
Epoch 2: saving model to SavedModels/CD_1P1_best_ft2_epoch002_val0.0588.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 4162s 7s/step - accuracy: 0.9845 - loss: 0.0428 - val_accuracy: 0.9876 - val_loss: 0.0588
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9857 - loss: 0.0388
Epoch 3: saving model to SavedModels/CD_1P1_best_ft2_epoch003_val0.0598.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3508s 6s/step - accuracy: 0.9857 - loss: 0.0388 - val_accuracy: 0.9878 - val_loss: 0.0598


In [ ]:
import json

model_name = 'CD1 P1_FT2_continue'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData'
filepath = f"SavedModels\\{model_name}.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)